<a href="https://colab.research.google.com/github/charlesy1018/HKSTOCK-market-daily-check/blob/main/HK_Stock_analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install statsmodels
!pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Preparing list to analysis**


In [2]:
import pandas as pd
import numpy as np 
import yfinance as yf

from scipy.stats import linregress
from datetime import datetime

In [3]:
url='https://raw.githubusercontent.com/charlesy1018/HKSTOCK-market-daily-check/main/HKSTOCK.csv'
stock_ticker=pd.read_csv(url)
print(stock_ticker)

     Symbol            Name
0   0001.HK    CKH HOLDINGS
1   0002.HK    CLP HOLDINGS
2   0003.HK  HK & CHINA GAS
3   0005.HK   HSBC HOLDINGS
4   0006.HK    POWER ASSETS
..      ...             ...
64  6862.HK        HAIDILAO
65  9618.HK           JD-SW
66  9633.HK   NONGFU SPRING
67  9988.HK         BABA-SW
68  9999.HK          NTES-S

[69 rows x 2 columns]


In [4]:
ticker_list = list(stock_ticker.Symbol)

**Retreving Data from yfinance**

In [5]:
today = str(datetime.now().date())

def getdata(tickers):
  data = []
  for ticker in tickers:
    data.append(yf.download(ticker,start='2020-01-01',end=today).reset_index())
  
  return data

In [6]:
stock_data = getdata(ticker_list)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
#Assigning symbol into the list of dataframe
##e.g:stock_data[0]['Symbol']=ticker_list[0]

for i in range(len(stock_data)):
  stock_data[i]['Symbol']=ticker_list[i]




In [8]:
## concat all the item from the list to a big dataframe
df=pd.concat(stock_data)
df



#method 2 to concat range(len(list))<< get the index number of each dataframe then do concatnation
#df=pd.DataFrame()
#for x in range(len(stock_data)): 
#  df=pd.concat([df,stock_data[x]])


,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2020-01-02,74.500000,75.199997,74.300003,74.500000,65.464165,2294978,0001.HK
1,2020-01-03,75.000000,75.250000,73.849998,74.099998,65.112671,2836418,0001.HK
2,2020-01-06,73.500000,74.650002,73.199997,73.550003,64.629387,3232362,0001.HK
3,2020-01-07,74.050003,74.300003,72.800003,73.050003,64.190025,4152215,0001.HK
4,2020-01-08,71.699997,72.449997,71.250000,72.150002,63.399189,4943850,0001.HK
...,...,...,...,...,...,...,...,...
497,2022-06-20,144.800003,144.899994,135.300003,141.000000,141.000000,25145696,9999.HK
498,2022-06-21,141.500000,143.600006,140.199997,142.699997,142.699997,11667156,9999.HK
499,2022-06-22,146.000000,146.000000,138.500000,139.199997,139.199997,8519701,9999.HK
500,2022-06-23,142.300003,145.300003,141.399994,142.399994,142.399994,10034624,9999.HK


In [9]:
df['Pre_Close']=df['Adj Close'].shift(1)
df['year']=df.Date.dt.year
df['week']=df.Date.dt.isocalendar().week
df['min_year']=df['year'].min()
df['year_rebase']=df.year - df.min_year
df['week_rebase']= 52*df.year_rebase + df.week
df.set_index('Date',inplace=True)


**Calculate SMA**

In [10]:
#Create Function Calculating SMA
def calculate_SMA(df,sma):
  df['SMA'+str(sma)]=df.groupby('Symbol')['Adj Close'].apply(lambda x: x.rolling(sma).mean())
  return df['SMA'+str(sma)]

In [11]:
calculate_SMA(df,20)
calculate_SMA(df,50)
calculate_SMA(df,200)

df

,Open,High,Low,Close,Adj Close,Volume,Symbol,Pre_Close,year,week,min_year,year_rebase,week_rebase,SMA20,SMA50,SMA200
Date,,,,,,,,,,,,,,,,
2020-01-02,74.500000,75.199997,74.300003,74.500000,65.464165,2294978,0001.HK,NaN,2020,1,2020,0,1,NaN,NaN,NaN
2020-01-03,75.000000,75.250000,73.849998,74.099998,65.112671,2836418,0001.HK,65.464165,2020,1,2020,0,1,NaN,NaN,NaN
2020-01-06,73.500000,74.650002,73.199997,73.550003,64.629387,3232362,0001.HK,65.112671,2020,2,2020,0,2,NaN,NaN,NaN
2020-01-07,74.050003,74.300003,72.800003,73.050003,64.190025,4152215,0001.HK,64.629387,2020,2,2020,0,2,NaN,NaN,NaN
2020-01-08,71.699997,72.449997,71.250000,72.150002,63.399189,4943850,0001.HK,64.190025,2020,2,2020,0,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-20,144.800003,144.899994,135.300003,141.000000,141.000000,25145696,9999.HK,151.100006,2022,25,2020,2,129,160.473697,151.613366,151.551539
2022-06-21,141.500000,143.600006,140.199997,142.699997,142.699997,11667156,9999.HK,141.000000,2022,25,2020,2,129,159.916698,151.342585,151.565660
2022-06-22,146.000000,146.000000,138.500000,139.199997,139.199997,8519701,9999.HK,142.699997,2022,25,2020,2,129,159.329643,151.101765,151.554792


**Calculate Relative Strength**

In [12]:
# Define ROC_week and ROC_day
def Calculate_ROC_week( df,periods=None):
  data=df.groupby(['Symbol','week_rebase']).last().reset_index(level=1)
  data['ROC_week_'+str(periods)]= data['Adj Close'].pct_change(periods=periods)
  data = data.groupby('Symbol').last().loc[:,'ROC_week_'+str(periods)].reset_index()
  return data


def Calculate_ROC_day(df,periods=None):
  df['ROC'+str(periods)]=df.groupby('Symbol')['Adj Close'].apply(lambda x: x.pct_change(periods=periods))
  return df['ROC'+str(periods)]


In [13]:
#Calculate The Relative Strength 2*ROC63 +ROC126 +ROC189 +ROC252
Calculate_ROC_day( df,periods=63)
Calculate_ROC_day( df,periods=126)
Calculate_ROC_day( df,periods=189)
Calculate_ROC_day( df,periods=252)


df['RS_line']=(2*df.ROC63)+df.ROC126+df.ROC189+df.ROC252

**Insert ROC26 Week**

In [14]:
#Add ROC26 week into dataframe
roc26_week = Calculate_ROC_week( df,periods=26)
df.reset_index(inplace=True)
df= df.merge(roc26_week,how='left',on='Symbol')
df.set_index('Date',inplace=True)

**Calculate the drawdown from the 52Week High**

In [15]:
#calculate 52weeks high from each symbol
week52 = df.groupby('Symbol')['High'].apply(lambda x:x.resample('w').max().rolling(52).max()).reset_index(level=0)
latest_52_week_high = week52.groupby('Symbol').last().rename(columns ={'High':'52_Week_High'})

#merging the 52week high data and calculate the drawdown
df.reset_index(inplace=True)
df = df.merge(latest_52_week_high,how='left',on='Symbol')
df.set_index('Date',inplace=True)
df['DD From 52Weeks High']= (df['Close']/df['52_Week_High'])-1



**Calculate if the SMA line are moving upward(check the Slope of the SMA line )**

In [16]:
from statsmodels.formula.api import ols

def calculate_slope_by_linreg(symbol, array, window):

    if (len(array) < window):
        return None

    if np.isnan(array).all():
        return None

    X = np.arange(0, window)
    Y = array[-window:]

    d = pd.DataFrame({"X": pd.Series(X), "Y": pd.Series(Y)})
    d = d.tail(window)

    model = ols("Y ~ X", data=d)
    result = model.fit()

    return result.params[1]

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [19]:
from sklearn.linear_model import LinearRegression

def calculate_slope(array, window):
  X = np.arange(0,window).reshape((-1,1))
  Y = array[-window:]

  model = LinearRegression().fit(X, Y)
  result = model.coef_[0]

  return result>0


In [20]:
df2.groupby("Symbol").apply(lambda x: calculate_slope(x.SMA20.values, window=50))

Symbol
0001.HK    False
0002.HK     True
0003.HK    False
0005.HK    False
0006.HK     True
           ...  
6862.HK     True
9618.HK     True
9633.HK     True
9988.HK    False
9999.HK     True
Length: 69, dtype: bool

**Building Summary**

In [21]:
df.reset_index(inplace=True)

In [22]:
latest= df.groupby('Symbol').last()

In [23]:
latest = latest.merge(stock_ticker.set_index('Symbol'),left_index=True, right_index=True)
latest.loc[:,'ROCWeek26_gt_0'] = (latest['ROC_week_26']>0)
latest['Price > 20']=(latest.Pre_Close>latest.SMA20)
latest['Price > 50']=(latest.Pre_Close>latest.SMA50)
latest['Price > 200']=(latest.Pre_Close>latest.SMA200)
latest['SMA 20 Trend up']=df.groupby("Symbol").apply(lambda x: calculate_slope(x.SMA20.values, window=50))
latest['SMA 50 Trend up']=df.groupby("Symbol").apply(lambda x: calculate_slope(x.SMA50.values, window=50))
latest['SMA 200 Trend up']=df.groupby("Symbol").apply(lambda x: calculate_slope(x.SMA200.values, window=50))
latest['20 > 50 > 200']=(latest.SMA20>latest.SMA50) & (latest.SMA50>latest.SMA200)

In [24]:
latest.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Pre_Close', 'year', 'week', 'min_year', 'year_rebase', 'week_rebase',
       'SMA20', 'SMA50', 'SMA200', 'ROC63', 'ROC126', 'ROC189', 'ROC252',
       'RS_line', 'ROC_week_26', '52_Week_High', 'DD From 52Weeks High',
       'Name', 'ROCWeek26_gt_0', 'Price > 20', 'Price > 50', 'Price > 200',
       'SMA 20 Trend up', 'SMA 50 Trend up', 'SMA 200 Trend up',
       '20 > 50 > 200'],
      dtype='object')

In [30]:
Output=latest.loc[:,['Name',
                     'Date',
                     'Pre_Close',
                     'RS_line',
                     'DD From 52Weeks High',
                     'ROCWeek26_gt_0',
                     'Price > 20',
                     'SMA 20 Trend up',
                     'Price > 50',
                     'SMA 50 Trend up',
                     'Price > 200',
                     'SMA 200 Trend up',
                     '20 > 50 > 200']]\
                .reset_index()\
                .set_index("Name")\
                .sort_values("RS_line",ascending= False)

In [31]:
Output

,Symbol,Date,Pre_Close,RS_line,DD From 52Weeks High,ROCWeek26_gt_0,Price > 20,SMA 20 Trend up,Price > 50,SMA 50 Trend up,Price > 200,SMA 200 Trend up,20 > 50 > 200
Name,,,,,,,,,,,,,
BYD COMPANY,1211.HK,2022-06-24,310.000000,1.730303,-0.044362,True,True,True,True,True,True,True,True
OOIL,0316.HK,2022-06-24,216.399994,1.596230,-0.238062,True,False,True,False,True,True,True,True
CNOOC,0883.HK,2022-06-24,10.000000,1.200044,-0.213716,True,False,True,False,True,True,True,True
CHINA OVERSEAS,0688.HK,2022-06-24,23.350000,1.072361,-0.153565,True,True,False,True,False,True,True,False
BOC HONG KONG,2388.HK,2022-06-24,31.049999,0.901169,-0.053435,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
COUNTRY GARDEN,2007.HK,2022-06-24,4.600000,-1.423623,-0.492291,False,True,False,False,False,False,False,False
CG SERVICES,6098.HK,2022-06-24,31.850000,-1.448016,-0.618545,False,True,False,True,False,False,False,False
XIAOMI-W,1810.HK,2022-06-24,11.840000,-1.536199,-0.573473,False,False,False,True,False,False,False,False


In [34]:
Output[Output.Symbol=='1211.HK']

,Symbol,Date,Pre_Close,RS_line,DD From 52Weeks High,ROCWeek26_gt_0,Price > 20,SMA 20 Trend up,Price > 50,SMA 50 Trend up,Price > 200,SMA 200 Trend up,20 > 50 > 200
Name,,,,,,,,,,,,,
BYD COMPANY,1211.HK,2022-06-24,310.0,1.730303,-0.044362,True,True,True,True,True,True,True,True


In [35]:
Output.to_csv('export.csv')